In [0]:
from google.colab import drive
from zipfile import ZipFile
import pandas as pd
import numpy as np
import datetime

# https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Loading Odds Data

Excel downloaded from: https://sportsbookonline.com



In [0]:
def get_season_outcomes(binary):
  # Download the Excel
  df = pd.read_excel(binary)
  season = str(start)+'-'+str(end)
  season = season[0:5]+season[7:9]
  games = []
  # (Date, Odds, Winner, Loser, Outcome, Point Diff)
  for i in range(0, len(df)-1, 2):
    year = start if df.iloc[i]['Date'] > 1000 else end
    # Two possible outcomes for every game
    g1 = [season, '{:04}'.format(df.iloc[i]['Date'])+str(year), df.iloc[i]['ML'], df.iloc[i]['Team'], df.iloc[i+1]['Team'], df.iloc[i]['Final'] > df.iloc[i+1]['Final'], df.iloc[i]['Final'] - df.iloc[i+1]['Final']]
    g2 = [season, '{:04}'.format(df.iloc[i]['Date'])+str(year), df.iloc[i+1]['ML'], df.iloc[i+1]['Team'], df.iloc[i]['Team'], df.iloc[i+1]['Final'] > df.iloc[i]['Final'], df.iloc[i+1]['Final'] - df.iloc[i]['Final']]
    # Add them both to the DF
    games.append(pd.DataFrame([g1], columns=['Season', 'Date', 'Odds', 'Winner', 'Loser', 'Outcome', 'Point Differential']))
    games.append(pd.DataFrame([g2], columns=['Season', 'Date', 'Odds', 'Winner', 'Loser', 'Outcome', 'Point Differential']))
  return pd.concat(games)

def get_all_seasons(path):
  seasons = []
  with ZipFile(path, 'r') as zipObj:
    names = zipObj.listnames()
    for name in names:
      seasons.append(get_season_outcomes(zipObj.open(name, 'r')))
  return pd.concat(seasons)

df = get_all_seasons()

In [0]:
def transform_dates(dates):
  new_dates = []
  for date in dates:
    new_date = date[0:2] + '/' + date[2:4] + '/' + date[4:8]
    new_date = datetime.datetime.strptime(new_date, '%m/%d/%Y')
    # https://stackoverflow.com/questions/441147/how-to-subtract-a-day-from-a-date
    new_date = new_date - datetime.timedelta(days=1)
    new_dates.append(new_date.strftime('%m/%d/%Y'))
  return new_dates

def transform_odds(odds):
  new_odds = []
  for odd in odds:
    if odd == 'NL':
      euro = 0.0
    else:
      euro = (odd / 100) + 1 if odd > 0 else (100 / abs(odd)) + 1
    new_odds.append(euro)
  return new_odds

def team_lookup(city):
  teams = {
      'Atlanta':'1610612737',
      'Boston':'1610612738',
      'Brooklyn':'1610612751',
      'NewJersey':'1610612751',
      'Charlotte':'1610612766',
      'Chicago':'1610612741',
      'Cleveland':'1610612739',
      'Dallas':'1610612742',
      'Denver':'1610612743',
      'Detroit':'1610612765',
      'GoldenState':'1610612744',
      'Houston':'1610612745',
      'Indiana':'1610612754',
      'LAClippers':'1610612746',
      'LALakers':'1610612747',
      'Memphis':'1610612763',
      'Miami':'1610612748',
      'Milwaukee':'1610612749',
      'Minnesota':'1610612750',
      'NewOrleans':'1610612740',
      'NewYork':'1610612752',
      'OklahomaCity':'1610612760',
      'Seattle':'1610612760',
      'Orlando':'1610612753',
      'Philadelphia':'1610612755',
      'Phoenix':'1610612756',
      'Portland':'1610612757',
      'Sacramento':'1610612758',
      'SanAntonio':'1610612759',
      'Toronto':'1610612761',
      'Utah':'1610612762',
      'Washington':'1610612764'
  }
  return teams[city.replace(' ', '')]

def transform_teams(teams):
  new_teams = []
  for team in teams:
    new_team = team_lookup(team)
    new_teams.append(new_team)
  return new_teams

df['Date'] = transform_dates(df['Date'])
df['Odds'] = transform_odds(df['Odds'])
df['Winner'] = transform_teams(df['Winner'])
df['Loser'] = transform_teams(df['Loser'])

In [0]:
df['Date'] = pd.to_datetime(df['Date'])

In [0]:
s1 = '11-15-2007'
e1 = '04-01-2008'
s2 = '11-15-2008'
e2 = '04-01-2009'
s3 = '11-15-2009'
e3 = '04-01-2010'
s4 = '11-15-2010'
e4 = '04-01-2011'
s5 = '01-15-2012' # LOCKOUT YR :(
e5 = '04-01-2012'
s6 = '11-15-2012'
e6 = '04-01-2013'
s7 = '11-15-2013'
e7 = '04-01-2014'
s8 = '11-15-2014'
e8 = '04-01-2015'
s9 = '11-15-2015'
e9 = '04-01-2016'
s10 = '11-15-2016'
e10 = '04-01-2017'
s11 = '11-15-2017'
e11 = '04-01-2018'
s12 = '11-15-2018'
e12 = '04-01-2019'
s13 = '11-15-2019'
e13 = '04-01-2020'
# https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values
mask = (((df['Date'] > s1) & (df['Date'] < e1)) | 
        ((df['Date'] > s2) & (df['Date'] < e2)) | 
        ((df['Date'] > s3) & (df['Date'] < e3)) | 
        ((df['Date'] > s4) & (df['Date'] < e4)) |
        ((df['Date'] > s5) & (df['Date'] < e5)) | 
        ((df['Date'] > s6) & (df['Date'] < e6)) | 
        ((df['Date'] > s7) & (df['Date'] < e7)) | 
        ((df['Date'] > s8) & (df['Date'] < e8)) |
        ((df['Date'] > s9) & (df['Date'] < e9)) | 
        ((df['Date'] > s10) & (df['Date'] < e10)) | 
        ((df['Date'] > s11) & (df['Date'] < e11)) | 
        ((df['Date'] > s12) & (df['Date'] < e12)) |
        ((df['Date'] > s13) & (df['Date'] < e13)))

df = df.loc[mask]

In [0]:
df.reset_index(drop=True).to_csv('matchups.csv')

In [0]:
df

,Season,Date,Odds,Winner,Loser,Outcome,Point Differential
0,2007-08,2007-11-16,2.750000,1610612757,1610612764,False,-19
0,2007-08,2007-11-16,1.487805,1610612764,1610612757,True,19
0,2007-08,2007-11-16,4.000000,1610612760,1610612766,False,-16
0,2007-08,2007-11-16,1.277778,1610612766,1610612760,True,16
0,2007-08,2007-11-16,1.434783,1610612762,1610612754,False,-20
...,...,...,...,...,...,...,...
0,2019-20,2020-01-25,1.714286,1610612752,1610612751,True,13
0,2019-20,2020-01-25,1.952381,1610612764,1610612737,False,-19
0,2019-20,2020-01-25,1.869565,1610612737,1610612764,True,19
0,2019-20,2020-01-25,2.300000,1610612754,1610612757,False,-10


In [0]:
!cp matchups.csv 'gdrive/My Drive/matchups.csv'